### Financial Crime & Suspicious Matter Reporting - LLM-based Agentic Solution

This sample code has been put together to demonstrate the capabilities of LLMs in being able to detect money laundering and other financial crime related activities. The example below demonstrates the use of LLM-based agents to identify suspicious activities from a call-centre log and draft a suspicious matter report.

The multi-agent example published by Microsoft has been used as a reference for this notebook - https://github.com/Azure-Samples/azureai-samples/blob/main/scenarios/Assistants/multi-agent/multi-agent.ipynb


In [164]:
# Uncomment line below to install dependencies; re-comment after installation.
# %pip install -r requirements.txt

In [165]:
import os
import time
from dotenv import load_dotenv
from openai import AzureOpenAI
from openai.types.beta import Thread
from openai.types.beta import Assistant

load_dotenv()

assistant_client = AzureOpenAI(
    api_key=os.getenv("GPT4_AZURE_OPENAI_KEY"),
    api_version=os.getenv("GPT4_AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv(
        "GPT4_AZURE_OPENAI_ENDPOINT"
    ),
)

assistant_deployment_name = os.getenv(
    "GPT4_DEPLOYMENT_NAME"
)

In [166]:
name_reviewer = "reviewer_assistant"
instructions_reviewer = """You are an assistant that understands guidelines published by AUSTRAC for detecting anti-money laundering and financial crime activities. Your job is to review the log from a call centre interaction between an agent and a customer and detect any suspicious activities. Generate a detailed summary with specific points on the suspicious activities."""

verbose_output = True

In [167]:
reviewer_assistant = assistant_client.beta.assistants.create(
    name=name_reviewer, instructions=instructions_reviewer, model=assistant_deployment_name
)

In [168]:
name_reportwriter = "reportwriter_assistant"
instructions_reportwriter = """You are an assistant that receives a list of suspicious activities and then drafts a Suspicious Matter Report (SMR) for it based on AUSTRAC's published guidelines. Ensure the report aligns with the format required by AUSTRAC."""

verbose_output = True

In [169]:
reportwriter_assistant = assistant_client.beta.assistants.create(
    name=name_reportwriter, instructions=instructions_reportwriter, model=assistant_deployment_name
)

In [170]:
name_approver = "approver_assistant"
instructions_approver = """You are an assistant that understands AUSTRAC's published guidelines for Suspicious Matter Reporting (SMR). Your job is to review a draft SMR and ensure it captures all suspicious activities that were previously identified."""

verbose_output = True

In [171]:
approver_assistant = assistant_client.beta.assistants.create(
    name=name_approver, instructions=instructions_approver, model=assistant_deployment_name
)

In [172]:
thread = assistant_client.beta.threads.create()

In [173]:
user_message = f"""
    << Add call-centre log here >>
    """

In [174]:
def run_assistant(assistant_id, thread_id, instructions):
    run = assistant_client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        instructions=instructions,
    )

    while True:
        run = assistant_client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

        if run.status=="completed":
            print("Done!")
            break
        else:
            print("Task executing ...")
            time.sleep(5)

In [175]:
def print_messages(thread_id):
    messages = assistant_client.beta.threads.messages.list(
      thread_id=thread_id
    )

    for msg in messages.data:
        for content_item in msg.content:
            print(content_item.text.value)

In [176]:
run_assistant(
    reviewer_assistant.id, 
    thread.id, 
    f"Review the conversation log provided and draft a summary of suspicious activities - " + user_message
)

Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Done!


In [ ]:
print_messages(thread.id)

In [178]:
run_assistant(
    reportwriter_assistant.id, 
    thread.id, 
    f"Review the summary of suspicious activities written by the reviewer_assistant assitant and draft a Suspicious Matter Report (SMR)"
)

Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Done!


In [ ]:
print_messages(thread.id)

In [180]:
run_assistant(
    approver_assistant.id, 
    thread.id, 
    f"Check the Suspicious Matter Report (SMR) created by the reportwriter_assistant assistant and make sure all points mentioned in the summary of suspicious activities generated by the reviewer_assistant assitant are captured in the final report."
)

Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Done!


In [ ]:
print_messages(thread.id)

In [182]:
# Clean-up
response = assistant_client.beta.assistants.delete(reviewer_assistant.id)
response = assistant_client.beta.assistants.delete(reportwriter_assistant.id)